In [2]:
import pandas as pd

In [3]:
# Cargamos los archivos parquet
df1 = pd.read_parquet('consulta1.parquet')
games = pd.read_parquet('games.parquet')
reviews = pd.read_parquet('reviews.parquet')
items = pd.read_parquet('items.parquet')


In [4]:
#@app.get('/consulta1 - Developer', tags=['GET'])

def developer(desarrollador):
    """Devuelve cantidad de items y porcentaje de contenido free por año según empresa desarrolladora\n.
    """
    
    #filtrar juego por desarrollador
    filter_games = df1[df1['developer'] == desarrollador]

    # Agrupar por año de lanzamiento y calcular cantidad de items y juegos gratuitos
    filter_games = filter_games.groupby('release_date').agg(total_items=('content_id', 'count'), free_game=('price', lambda x: (x == 0.0).sum())).reset_index()
    
    # Calcular el porcentaje de contenido gratuito
    filter_games['Contenido Free'] = round((filter_games['free_game'] / filter_games['total_items']) * 100, 2)
    
    # Formatear los resultados
    año = list(filter_games['release_date'])
    cantidad_items = list(filter_games['total_items'])
    cont_free = list(filter_games['Contenido Free'])
    
    # Retornar el resultado en el formato solicitado
    return {'Año': año, 'Cantidad de items': cantidad_items, 'Contenido Free': cont_free}

In [5]:
developer('Kotoshiro')

{'Año': [2018], 'Cantidad de items': [1], 'Contenido Free': [0.0]}

In [13]:
def userdata(User_id: str):
    """Devuelve la cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items\n.
    """
    # Selección de columnas
    game = games[['content_id', 'price']]
    
    # Filtro por usuario
    user = items[items['user_id'] == User_id]

    if user.empty:
        return {'Usuario': User_id, 'Dinero gastado': 0.0, 'Horas juego': 0, '% de recomendacion': 0.0}

    # Calcular el porcentaje de recomendación
    if reviews.empty:
        recomend = 0.0
    else:
        recomend = round(reviews['review_pos'].sum() / (reviews['review_pos'].sum() + reviews['review_neg'].sum()) * 100, 2)

    # Cantidad de juegos por usuario
    cantidad_items = items[items['user_id'] == User_id]

    # Unión de los datos
    cantidad_items = cantidad_items.merge(game, left_on='item_id', right_on='content_id', how='inner')
    if cantidad_items.empty:
        return {'Usuario': User_id, 'Dinero gastado': 0.0, 'Horas juego': 0, '% de recomendacion': recomend}

    cantidad_items = cantidad_items.groupby('user_id').agg({'playtime_forever': 'sum', 'price': 'sum'}).reset_index()

    if cantidad_items.empty:
        return {'Usuario': User_id, 'Dinero gastado': 0.0, 'Horas juego': 0, '% de recomendacion': recomend}

    # Variables:
    usuario = cantidad_items['user_id'].iloc[0]
    tiempo = cantidad_items['playtime_forever'].iloc[0]
    dinero = cantidad_items['price'].iloc[0]

    return {'Usuario': usuario, 'Dinero gastado': round(dinero, 2), 'Horas juego': tiempo, '% de recomendacion': recomend}

In [17]:
userdata('76561198076909484')

{'Usuario': '76561198076909484',
 'Dinero gastado': 374.93,
 'Horas juego': 16649.0,
 '% de recomendacion': 82.51}

In [ ]:
def UserForGenre( genero : str ): 
#Debe devolver el usuario que acumula más horas jugadas para el género dado 
#y una lista de la acumulación de horas jugadas por año de lanzamiento.

Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [ ]:
def best_developer_year( año : int ):
#Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios
#para el año dado. (reviews.recommend = True y comentarios positivos)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [ ]:
def developer_reviews_analysis( desarrolladora : str ): 
# Según el desarrollador, se devuelve un diccionario con 
#el nombre del desarrollador como llave y 
#una lista con la cantidad total de registros de reseñas de usuarios 
#que se encuentren categorizados con un análisis de sentimiento 
#como valor positivo o negativo.

Ejemplo de retorno: {'Valve' : [Negative = 182, Positive = 278]}